Carteira baseada no número de Graham sobre o valor intrínseco de uma ação, foca em pegar empresas que sejam lucrativas e baratas: empresas operando a P/L menor que 15 e com P/VPA menor que 1,5. Multiplicando 15 x 1,5 temos o número 22,5 que nos indica a fórmula de graham. Valor Intrínseco de uma ação = $\sqrt{22,5 \times \text{LPA} \times \text{VPA}}$.

Critérios:


- Ter Lucro por ação maior que zero ,isto é, a empresa não pode estar com prejuízo atualmente
- Ter Valor Patrimonial por ação positivo, ou seja, a empresa não pode ter mais passivos (obrigações a pagar) que ativos (bens ou direitos a receber)
- Ter volume médio de negociação diário de no mínimo R$250.000,00
- Ter lucro líquido medio positivo em todos os últimos 5 exercícios.


In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from IPython.display import display, HTML

def b_print(df , n=30 , clean=True): #beauty print :)
    
    # from IPython.display import display, HTML

    # if clean : # remove tickers da mesma empresa, deixando a primeria ocorrencia
    #     df['prefixo'] = df['Papel'].astype(str).str[:4]
    #     df=df.drop_duplicates(subset='prefixo', keep='first')
    #     # df=df.drop('prefixo', axis=1) 
    
    display(HTML(df.head(n).to_html(index=False)))
    df = None


In [2]:
from DT_atualiza_settings import *
from DT_StatusInvest import SI
SI(mercado = 'Acoes' )

 ====== SI Acoes ===== 
 ====== Busca
 ====== Download 


In [3]:
import os
# Caminho do arquivo local
file_path = os.path.expanduser('~/GHub/Finance-playground/data/SI_Acoes.csv')
# URL para o arquivo online
file_url = 'https://raw.githubusercontent.com/BDonadelli/Finance-playground/refs/heads/main/data/SI_Acoes.csv'

# Verificar se o arquivo existe localmente
if os.path.exists(file_path):
    # Ler o arquivo local
    funds = pd.read_csv(file_path,sep=';' , decimal=',' ,thousands ='.' )
    print("Arquivo lido localmente.")
else:
    # Ler o arquivo a partir da URL
    funds = pd.read_csv(file_url,sep=';' , decimal=',' ,thousands ='.' )
    print("Arquivo lido da URL.")


Arquivo lido localmente.


In [4]:
funds.columns

Index(['TICKER', 'PRECO', 'DY', 'P/L', 'P/VP', 'P/ATIVOS', 'MARGEM BRUTA',
       'MARGEM EBIT', 'MARG. LIQUIDA', 'P/EBIT', 'EV/EBIT',
       'DIVIDA LIQUIDA / EBIT', 'DIV. LIQ. / PATRI.', 'PSR', 'P/CAP. GIRO',
       'P. AT CIR. LIQ.', 'LIQ. CORRENTE', 'ROE', 'ROA', 'ROIC',
       'PATRIMONIO / ATIVOS', 'PASSIVOS / ATIVOS', 'GIRO ATIVOS',
       'CAGR RECEITAS 5 ANOS', 'CAGR LUCROS 5 ANOS', ' LIQUIDEZ MEDIA DIARIA',
       ' VPA', ' LPA', ' PEG Ratio', ' VALOR DE MERCADO'],
      dtype='object')

In [5]:
fundsSI =  funds[ (funds[' LIQUIDEZ MEDIA DIARIA'] > 300000) &
                  (funds[' LPA'] > 0) & 
                  (funds[' VPA'] > 0) & 
                  (funds['CAGR LUCROS 5 ANOS'] > 0) ]
fundsSI

fundsSI['valor intrinseco'] = np.round(np.sqrt(22.5 * fundsSI[' LPA'] * fundsSI[' VPA']),2)
fundsSI['VALOR DE MERCADO (em B)'] = fundsSI[' VALOR DE MERCADO']  / 1e9
fundsSI['LIQUIDEZ MEDIA DIARIA (em M)'] = fundsSI[' LIQUIDEZ MEDIA DIARIA'] / 1e6
fundsSI['Delta (%)'] = np.round((fundsSI['valor intrinseco'] / fundsSI['PRECO'] -1)*100,2)
fundsSI["Rank"]   = fundsSI['Delta (%)'].rank(ascending=True, method="min")
fundsSI.sort_values(by="Rank", ascending=False, inplace=True)
fundsSI.reset_index(inplace=True)
fundsSI.index = fundsSI.index + 1

colunas_exibidas = ['TICKER','PRECO' , 'CAGR LUCROS 5 ANOS' ,'LIQUIDEZ MEDIA DIARIA (em M)',
                 ' VPA',' LPA' ,  'VALOR DE MERCADO (em B)',
                'valor intrinseco' , 'Delta (%)' ,'P/L', 'DIVIDA LIQUIDA / EBIT' ]

b_print(fundsSI[colunas_exibidas])

TICKER,PRECO,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA (em M),VPA,LPA,VALOR DE MERCADO (em B),valor intrinseco,Delta (%),P/L,DIVIDA LIQUIDA / EBIT
LIGT3,5.11,12.76,3.780219,15.00,6.50,1.903758,46.84,816.63,0.79,4.64
COGN3,2.90,36.40,56.533470,6.70,0.59,5.442158,9.43,225.17,4.90,3.62
JHSF3,5.93,28.82,20.154020,8.79,1.68,4.040244,18.23,207.42,3.54,1.99
SOMA3,5.95,80.87,99.716529,7.40,2.01,4.669247,18.29,207.39,2.96,-0.45
MTRE3,3.50,12.44,1.055198,9.38,0.54,0.370212,10.68,205.14,6.45,5.22
LOGG3,21.28,32.59,4.328997,42.57,4.21,1.869643,63.50,198.40,5.06,3.92
TASA4,4.55,22.22,1.404740,9.15,0.85,0.661905,13.23,190.77,5.35,4.24
EUCA4,16.98,31.55,0.399516,29.45,2.79,1.709271,43.00,153.24,6.08,1.21
USIM3,4.57,12.02,2.465442,19.37,0.30,5.764919,11.43,150.11,15.23,0.96
PRIO3,35.34,75.51,164.890351,29.21,11.81,31.676874,88.10,149.29,2.99,4.75


outros criterios adicionais

P/L > 0 e DIVIDA LIQUIDA / EBIT < 3

In [6]:
'''
P/L > 0 e DIVIDA LIQUIDA / EBIT < 3
'''
fundsSI.fillna(0,inplace=True) ## bancos tem NaN
b_print(fundsSI[(fundsSI['P/L'] > 0) & (fundsSI['DIVIDA LIQUIDA / EBIT'] < 3) ][colunas_exibidas])

TICKER,PRECO,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA (em M),VPA,LPA,VALOR DE MERCADO (em B),valor intrinseco,Delta (%),P/L,DIVIDA LIQUIDA / EBIT
JHSF3,5.93,28.82,20.154020,8.79,1.68,4.040244,18.23,207.42,3.54,1.99
SOMA3,5.95,80.87,99.716529,7.40,2.01,4.669247,18.29,207.39,2.96,-0.45
EUCA4,16.98,31.55,0.399516,29.45,2.79,1.709271,43.00,153.24,6.08,1.21
USIM3,4.57,12.02,2.465442,19.37,0.30,5.764919,11.43,150.11,15.23,0.96
USIM5,4.64,12.02,32.390763,19.37,0.30,5.764919,11.43,146.34,15.46,0.96
ISAE4,23.98,13.00,36.288440,31.27,4.93,17.724237,58.90,145.62,4.87,2.57
ALLD3,8.48,2.08,1.500784,15.81,1.18,0.804144,20.49,141.63,7.21,1.21
AZZA3,25.00,37.42,43.060292,40.39,3.85,5.162245,59.15,136.60,6.50,2.83
BBAS3,20.71,2.74,430.401559,31.76,3.27,118.685573,48.34,133.41,6.32,0.00
SAPR4,7.01,15.94,3.673558,7.80,1.50,10.862545,16.22,131.38,4.68,2.06


remove empresas repetidas, mantem primeira ocorrencia

In [7]:
fundsSI['prefixo'] = fundsSI['TICKER'].str[:4]
fundsSI.drop_duplicates(subset='prefixo', keep='first').head(20)
df_limpo = fundsSI.drop_duplicates(subset='prefixo', keep='first')
df_limpo = df_limpo.drop('prefixo', axis=1) 
b_print(df_limpo[colunas_exibidas])

TICKER,PRECO,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA (em M),VPA,LPA,VALOR DE MERCADO (em B),valor intrinseco,Delta (%),P/L,DIVIDA LIQUIDA / EBIT
LIGT3,5.11,12.76,3.780219,15.00,6.50,1.903758,46.84,816.63,0.79,4.64
COGN3,2.90,36.40,56.533470,6.70,0.59,5.442158,9.43,225.17,4.90,3.62
JHSF3,5.93,28.82,20.154020,8.79,1.68,4.040244,18.23,207.42,3.54,1.99
SOMA3,5.95,80.87,99.716529,7.40,2.01,4.669247,18.29,207.39,2.96,-0.45
MTRE3,3.50,12.44,1.055198,9.38,0.54,0.370212,10.68,205.14,6.45,5.22
LOGG3,21.28,32.59,4.328997,42.57,4.21,1.869643,63.50,198.40,5.06,3.92
TASA4,4.55,22.22,1.404740,9.15,0.85,0.661905,13.23,190.77,5.35,4.24
EUCA4,16.98,31.55,0.399516,29.45,2.79,1.709271,43.00,153.24,6.08,1.21
USIM3,4.57,12.02,2.465442,19.37,0.30,5.764919,11.43,150.11,15.23,0.96
PRIO3,35.34,75.51,164.890351,29.21,11.81,31.676874,88.10,149.29,2.99,4.75


### outra estória de porque 22,5

O número 22,5 na Fórmula de Graham é um fator de ponderação que tem um propósito específico. Essa constante foi escolhida por Graham pra ajustar a avaliação do preço justo de uma ação com base na taxa de crescimento anual esperada da empresa.

O número 22,5 é o resultado da multiplicação de 8,5 por 2,65 (8,5 x 2,65 = 22,5). O número 8,5 é a base que Graham considerou razoável pra uma empresa com taxa de crescimento zero, ou seja, uma empresa que não cresce. Já o número 2,65 representa a média do retorno exigido pelos investidores no mercado de ações durante a época de Graham, que era de aproximadamente 4,4% acima da taxa de retorno dos títulos do Tesouro dos Estados Unidos. O fator 22,5 ajuda a ajustar o preço justo com base no crescimento da empresa e na expectativa de retorno dos investidores. Esse ajuste garante que a Fórmula de Graham considere a taxa de crescimento anual esperada e reflita uma avaliação mais realista do preço justo de uma ação.

# Stocks

In [8]:
import os
# Caminho do arquivo local /home/yair/GHub/Codigos-em-financas/data/SI_Stocks.csv
file_path = os.path.expanduser('/home/yair/GHub/Finance-playground/data/SI_Stocks.csv')
# URL para o arquivo online
file_url = 'https://raw.githubusercontent.com/BDonadelli/Finance-playground/refs/heads/main/data/SI_Stocks.csv'

# Verificar se o arquivo existe localmente
if os.path.exists(file_path):
    # Ler o arquivo local
    funds = pd.read_csv(file_path,sep=';' , decimal=',' ,thousands ='.' )
    print("Arquivo lido localmente.")
else:
    # Ler o arquivo a partir da URL
    funds = pd.read_csv(file_url,sep=';' , decimal=',' ,thousands ='.' )
    print("Arquivo lido da URL.")


Arquivo lido localmente.


In [9]:
funds

,TICKER,PRECO,DY,P/L,P/VP,P/ATIVOS,MARGEM BRUTA,MARGEM EBIT,MARG. LIQUIDA,P/EBIT,EV/EBIT,DIVIDA LIQUIDA / EBIT,DIV. LIQ. / PATRI.,PSR,P/CAP. GIRO,P. AT CIR. LIQ.,LIQ. CORRENTE,ROE,ROA,ROIC,PATRIMONIO / ATIVOS,PASSIVOS / ATIVOS,GIRO ATIVOS,CAGR RECEITAS 5 ANOS,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA,VPA,LPA,PEG Ratio,VALOR DE MERCADO
0,A,127.09,0.58,29.59,5.67,2.95,13.01,5.77,17.66,23.89,96.39,4.76,0.29,5.32,15.29,-4.53,2.25,18.92,9.86,2.45,0.52,0.48,0.56,4.75,3.77,NaN,22.42,4.24,NaN,3.610178e+10
1,AA,33.60,0.60,8.66,1.42,0.58,6.04,3.00,7.86,5.62,28.29,5.58,0.35,0.68,4.09,-0.91,1.65,16.37,6.70,0.31,0.41,0.58,0.85,2.66,NaN,NaN,23.70,3.88,-0.03,8.699117e+09
2,AACG,2.33,NaN,-7.11,3.22,1.02,51.68,-31.49,-32.34,-7.31,-6.70,0.61,-0.27,2.30,-2.20,-1.19,0.23,-45.26,-14.39,-39.89,0.32,0.68,0.44,21.21,NaN,NaN,0.72,-0.33,-0.18,7.368508e+07
3,AACQ,9.99,NaN,-9.48,180.94,1.25,0.00,0.00,-360.49,NaN,NaN,NaN,-0.07,34.17,2094.26,-1.25,3.56,-1909.00,-13.16,NaN,0.01,0.12,0.04,NaN,NaN,NaN,0.06,-1.05,0.02,9.047194e+08
4,AACQU,11.05,NaN,-10.48,200.14,1.38,0.00,0.00,-360.49,NaN,NaN,NaN,-0.07,37.79,2316.47,-1.38,3.56,-1909.00,-13.16,NaN,0.01,0.12,0.04,NaN,NaN,NaN,0.06,-1.05,0.02,9.047194e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5265,ZUO,10.02,NaN,-20.82,8.36,1.80,67.42,-10.54,-16.34,-32.28,-28.30,3.99,-1.03,3.40,3.65,-8.80,2.62,-40.16,-8.63,-8.89,0.21,0.79,0.53,349.59,NaN,NaN,1.20,-0.48,0.40,1.540242e+09
5266,ZVO,0.09,NaN,-0.08,0.47,0.04,29.11,-18.74,-20.58,-0.09,0.67,0.76,-4.11,0.02,-1.38,-0.07,0.95,-593.02,-45.85,-541.17,0.08,0.92,2.23,-12.86,NaN,NaN,0.19,-1.15,0.00,3.078549e+06
5267,ZYME,16.05,NaN,-16.30,3.59,2.94,25.26,-14.48,-59.96,-17.82,-51.70,15.82,-0.84,9.78,4.66,-10.70,7.69,-22.03,-18.04,-7.19,0.82,0.18,0.30,20.90,NaN,NaN,4.47,-0.98,0.47,1.201254e+09
5268,ZYNE,1.30,NaN,-1.87,0.00,0.00,NaN,NaN,NaN,-8.75,4478.28,4487.03,-1.24,NaN,0.00,-0.06,3.43,-0.13,-0.09,-0.03,0.72,0.28,0.00,NaN,NaN,NaN,538.03,-0.70,4.43,7.012122e+07


In [10]:
funds.columns

Index(['TICKER', 'PRECO', 'DY', 'P/L', 'P/VP', 'P/ATIVOS', 'MARGEM BRUTA',
       'MARGEM EBIT', 'MARG. LIQUIDA', 'P/EBIT', 'EV/EBIT',
       'DIVIDA LIQUIDA / EBIT', 'DIV. LIQ. / PATRI.', 'PSR', 'P/CAP. GIRO',
       'P. AT CIR. LIQ.', 'LIQ. CORRENTE', 'ROE', 'ROA', 'ROIC',
       'PATRIMONIO / ATIVOS', 'PASSIVOS / ATIVOS', 'GIRO ATIVOS',
       'CAGR RECEITAS 5 ANOS', 'CAGR LUCROS 5 ANOS', ' LIQUIDEZ MEDIA DIARIA',
       ' VPA', ' LPA', ' PEG Ratio', ' VALOR DE MERCADO'],
      dtype='object')

In [11]:
fundsSI =  funds[ (funds[' LPA'] > 1) & 
                  (fundsSI['DIVIDA LIQUIDA / EBIT'] < 4) & 
                  (funds[' VPA'] > 0) 
                #   (funds['CAGR LUCROS 5 ANOS'] > 0) 
                  ]
b_print(fundsSI)

TICKER,PRECO,DY,P/L,P/VP,P/ATIVOS,MARGEM BRUTA,MARGEM EBIT,MARG. LIQUIDA,P/EBIT,EV/EBIT,DIVIDA LIQUIDA / EBIT,DIV. LIQ. / PATRI.,PSR,P/CAP. GIRO,P. AT CIR. LIQ.,LIQ. CORRENTE,ROE,ROA,ROIC,PATRIMONIO / ATIVOS,PASSIVOS / ATIVOS,GIRO ATIVOS,CAGR RECEITAS 5 ANOS,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA,VPA,LPA,PEG Ratio,VALOR DE MERCADO
AAPL,239.01,0.43,35.96,54.23,10.77,46.68,7.25,24.30,27.56,122.13,1.56,0.70,8.74,-191.63,-17.08,0.87,150.81,29.95,-0.39,0.20,0.80,1.23,8.49,11.15,NaN,4.41,6.65,1.41,3.569812e+12
AAWW,102.48,NaN,0.02,957.44,438.20,25591.50,5305.22,3776.40,0.02,-4.35,-4.36,-252517.36,0.88,-63287.62,-526.68,0.96,4119089.58,1885228.67,2849874.03,0.46,0.54,499.21,16.10,9.75,NaN,0.11,4408.90,0.00,2.934330e+09
ABB,37.12,NaN,13.49,5.68,1.80,0.00,0.00,8.87,NaN,NaN,NaN,0.09,1.20,89.85,-3.35,1.04,42.07,13.31,NaN,0.32,0.68,1.50,3.17,2.26,NaN,6.54,2.75,-2.86,6.900207e+10
ABC,179.95,NaN,25.91,52.69,0.59,3.47,0.97,0.72,19.20,21.13,1.93,5.29,0.19,-6.24,-1.79,0.88,203.31,2.28,27.19,0.01,0.99,3.16,9.27,36.05,NaN,3.42,6.94,-1.07,3.614368e+10
ABCB,73.59,0.75,12.96,1.29,0.19,0.00,32.51,94.23,9.33,16.52,-21.05,-0.73,3.07,NaN,-0.19,NaN,9.99,1.47,0.44,0.15,0.85,0.02,14.90,17.31,NaN,56.85,5.68,0.50,5.071153e+09
ABEO,5.66,NaN,5.06,1.77,1.18,0.00,-2052.75,14310.50,3.77,-10.14,25.12,-1.26,723.87,1.46,-20.72,6.73,34.99,23.25,-12.95,0.66,0.34,0.00,NaN,NaN,NaN,3.20,1.12,-0.05,2.895469e+08
ABEV,2.36,2.69,0.01,2.11,1.36,51.55,16894754.57,16092445.36,0.01,0.01,0.00,-0.19,2294.62,29.32,-1.92,1.19,14773.65,9520.71,12218.27,0.64,0.36,0.00,2.06,-4.53,NaN,1.12,165.51,0.00,3.679762e+10
ABG,242.08,NaN,8.81,1.26,0.47,17.05,1.39,3.13,19.85,37.69,17.84,1.13,0.28,6.55,-0.68,1.30,14.30,5.33,0.73,0.37,0.63,1.70,18.98,18.47,NaN,192.12,27.47,0.25,4.759099e+09
ABM,44.85,1.09,24.09,1.53,0.53,25.64,0.98,1.95,10.43,50.21,17.24,0.80,0.32,4.38,-0.84,1.49,9.25,3.21,0.60,0.35,0.65,1.65,5.16,-8.57,NaN,29.41,2.72,1.31,2.792271e+09
ABMD,381.02,NaN,64.40,11.16,10.13,61.12,18.85,24.84,84.86,80.81,-4.05,-0.53,16.00,18.50,-27.11,7.93,17.32,15.72,8.32,0.91,0.09,0.63,18.30,21.24,NaN,34.15,5.92,0.82,1.718065e+10


In [12]:
fundsSI['valor intrinseco'] = np.round(np.sqrt(22.5 * fundsSI[' LPA'] * fundsSI[' VPA']),2)
fundsSI['VALOR DE MERCADO (em B)'] = fundsSI[' VALOR DE MERCADO']  / 1e9
# fundsSI['LIQUIDEZ MEDIA DIARIA (em M)'] = fundsSI[' LIQUIDEZ MEDIA DIARIA'] / 1e6
fundsSI['Delta (%)'] = np.round((fundsSI['valor intrinseco'] / fundsSI['PRECO'] -1)*100,2)
fundsSI["Rank"]   = fundsSI['Delta (%)'].rank(ascending=True, method="min")
fundsSI.sort_values(by="Rank", ascending=False, inplace=True)
fundsSI.reset_index(inplace=True)
fundsSI.index = fundsSI.index + 1

colunas_exibidas = ['TICKER','PRECO' , 'CAGR LUCROS 5 ANOS' ,
                 ' VPA',' LPA' ,  'VALOR DE MERCADO (em B)',
                'valor intrinseco' , 'Delta (%)' ,'P/L', 'DIVIDA LIQUIDA / EBIT' ]

b_print(fundsSI[colunas_exibidas])

TICKER,PRECO,CAGR LUCROS 5 ANOS,VPA,LPA,VALOR DE MERCADO (em B),valor intrinseco,Delta (%),P/L,DIVIDA LIQUIDA / EBIT
ABEV,2.36,-4.53,1.12,165.51,36.797615,64.58,2636.44,0.01,0.00
ACIC.WS,1.68,NaN,6.05,1.67,0.525601,15.08,797.62,1.00,-2.67
AFGE,18.97,-0.22,54.07,9.15,11.569749,105.51,456.19,2.07,4.01
ACGLO,21.72,22.01,61.48,9.86,32.798514,116.79,437.71,2.20,2.08
AFGC,20.49,-0.22,54.07,9.15,11.569749,105.51,414.93,2.24,4.01
AFGD,22.10,-0.22,54.07,9.15,11.569749,105.51,377.42,2.42,4.01
AFGB,23.05,-0.22,54.07,9.15,11.569749,105.51,357.74,2.52,4.01
AEPPZ,47.26,9.08,55.92,6.83,57.367201,92.70,96.15,6.92,39.33
AEPPL,52.61,9.08,55.92,6.83,57.367201,92.70,76.20,7.70,39.33
AC,30.92,2.50,43.27,2.54,0.653578,49.73,60.83,12.15,-204.73
